In [1]:
# using StaticArrays
using RigidBodyDynamics
using MeshCatMechanisms
# using LinearAlgebra
using MeshCat

┌ Warning: attempting to remove probably stale pidfile
│   path = /Users/simondeligne/.jlassetregistry.lock
└ @ Pidfile /Users/simondeligne/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260


# Load mechanism

In [8]:
# urdf_path = "deps/biped_robot.urdf"
# urdf_path = "deps/ZMP_2DBipedRobot_noContacts.urdf"
# urdf_path = "deps/double_pendulum_obst.urdf"
# urdf_path = "deps/urdf/full_robot.urdf"
# urdf_path = "deps/robot.urdf"
# urdf_path = "deps/doublependulum_new_knee_fixed.urdf"
urdf_path = "../utils/urdfs/single_pendulum_hip/robot.urdf"
urdf_path = "../utils/urdfs/double_pendulum_right/robot.urdf"
# urdf_path = "../utils/urdfs/bipedal_robot/robot.urdf"
# urdf_path = "../utils/urdfs/bipedal_robot_fixed/robot.urdf"
# urdf_path = "../utils/urdfs/robot.urdf"
# urdf_path = "deps/ZMP_3DBipedRobot.urdf"
robot = parse_urdf(Float64, urdf_path)
remove_fixed_tree_joints!(robot)

Spanning tree:
Vertex: world (root)
  Vertex: leg_top_full_asmb, Edge: hip_right
    Vertex: leg_bottom_asmb, Edge: knee_right
No non-tree joints.

In [3]:
include("../last_mtk_version/model_urdf.jl")
using RigidBodyDynamics
robot = get_mechanism(;wanted_mech = "double_pendulum")


Spanning tree:
Vertex: world (root)
  Vertex: leg_top_full_asmb, Edge: hip_right
    Vertex: leg_bottom_asmb, Edge: knee_right
No non-tree joints.

In [4]:
joints(robot)

2-element Vector{Joint{Float64, JT} where JT<:JointType{Float64}}:
 Joint "hip_right": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "knee_right": Revolute joint with axis [0.0, 0.0, 1.0]

# Add ground

In [ ]:
frame = root_frame(robot)
ground = HalfSpace3D(Point3D(frame, 0., 0., -5), FreeVector3D(frame, 0., 0., 1.))
add_environment_primitive!(robot, ground)

# Visualisation

In [ ]:
vis = MechanismVisualizer(robot, URDFVisuals(urdf_path));

# Define and set the State

In [5]:
state = MechanismState(robot)

MechanismState{Float64, Float64, Float64, …}(…)

In [10]:
velocity(state)

5-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
mass_matrix(state)

5×5 LinearAlgebra.Symmetric{Float64, Matrix{Float64}}:
 0.0185639   0.0         0.00464347  0.0         0.0
 0.0         0.0185639   0.0         0.00464347  0.0
 0.00464347  0.0         0.00187312  0.0         0.0
 0.0         0.00464347  0.0         0.00187312  0.0
 0.0         0.0         0.0         0.0         0.0

In [69]:
SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}

SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}

In [44]:
# Let's first set the configurations and velocities of the joints:
#free_joint, shoulder_left, shoulder_right, elbow_left, elbow_right = joints(robot)
# hip_right, hip_left, knee_right, knee_left = joints(robot)
foot, knee_left, hip_left, hip_right, knee_right = joints(robot)
# hip_right = joints(robot)
# hip_right, knee_right = joints(robot)

5-element Vector{Joint{Float64, JT} where JT<:JointType{Float64}}:
 Joint "foot": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "knee_left": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "hip_left": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "hip_right": Revolute joint with axis [0.0, 0.0, 1.0]
 Joint "knee_right": Revolute joint with axis [0.0, 0.0, 1.0]

In [55]:
# set_configuration!(state, hip_right, pi/2)
# set_configuration!(state, knee_right, pi/2)
# set_configuration!(state, hip_left, 0)
# set_configuration!(state, knee_left, pi/2)

set_configuration!(state, hip_right, pi/10)
set_configuration!(state, knee_right, pi/10)
set_configuration!(state, hip_left, pi/10)
set_configuration!(state, knee_left, pi/10)


# set_configuration!(state, hip_right, 0)
# set_configuration!(state, knee_right, 0)
# set_configuration!(state, hip_left, 0)
# set_configuration!(state, knee_left, 0)

set_configuration!(state, foot, 0)

# set_velocity!(state, hip_right, 10)
# set_velocity!(state, knee_right, 0)
# set_velocity!(state, hip_left, 0)
# set_velocity!(state, knee_left, 0)
zero_velocity!(state)
# **Important**: a `MechanismState` contains cache variables that depend on the configurations and velocities of the joints. These need to be invalidated when the configurations and velocities are changed. To do this, call
setdirty!(state)

q = configuration(state)
v = velocity(state)
print("q=$q\nv=$v")

q=[0.0, 0.3141592653589793, 0.3141592653589793, 0.3141592653589793, 0.3141592653589793]
v=[0.0, 0.0, 0.0, 0.0, 0.0]

In [142]:
v̇ = similar(velocity(state))
v̇[hip_right][1] = 0.
v̇[knee_right][1] = 0.
τ_dynamics = -inverse_dynamics(state, v̇)

2-element Vector{Float64}:
 -0.5967035268827747
 -0.2770151409765261

In [143]:
v̇ = copy(velocity(state))
v̇[1] = 0.
v̇[2] = 0.
τ_dynamics = -inverse_dynamics(state, v̇)

2-element Vector{Float64}:
 -0.5967035268827747
 -0.2770151409765261

In [ ]:
set_configuration!(vis, configuration(state))

In [22]:
set_configuration!(state, [0, pi/2]) # hip, knee
# set_configuration!(actual_state, [pi/2])

zero_velocity!(state)

v̇ = similar(velocity(state))
fill!(v̇, 0.0)
τ_dynamics = -inverse_dynamics(state, v̇)
C = dynamics_bias(state)

2-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.13506931270115186
 0.1350413970002453

# Simulation

In [23]:
ts, qs, vs = simulate(state, 1., Δt = 1e-3);

In [18]:
mvis = MechanismVisualizer(robot, URDFVisuals(urdf_path))
animation = Animation(mvis, ts, qs)
setanimation!(mvis, animation)

# Create a MechanismVisualizer and visualize
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /Users/simondeligne/.julia/packages/MeshCat/oC0sL/src/visualizer.jl:73
